Data and meta-data available [here](http://download.geonames.org/export/dump/)

In [2]:
import sys,os,re,csv,urllib2
sys.path.append('/usr/local/lib/python2.7/dist-packages/')
import pymysql
import pandas as pd
from secrets import *
import BeautifulSoup as bs4

### Grab files

In [11]:
soup=bs4.BeautifulSoup(urllib2.urlopen('http://download.geonames.org/export/dump/').read())

In [16]:
for link in soup.findAll('a'):
    try:
        if re.search(r'[A-Z]{2,2}'+'\.zip',link.get('href')):
            print(link.get('href'))
            os.system('wget http://download.geonames.org/export/dump/'+link.get('href'))
    except:
        pass
    

AD.zip
AE.zip
AF.zip
AG.zip
AI.zip
AL.zip
AM.zip
AO.zip
AQ.zip
AR.zip
AS.zip
AT.zip
AU.zip
AW.zip
AX.zip
AZ.zip
BA.zip
BB.zip
BD.zip
BE.zip
BF.zip
BG.zip
BH.zip
BI.zip
BJ.zip
BL.zip
BM.zip
BN.zip
BO.zip
BQ.zip
BR.zip
BS.zip
BT.zip
BV.zip
BW.zip
BY.zip
BZ.zip
CA.zip
CC.zip
CD.zip
CF.zip
CG.zip
CH.zip
CI.zip
CK.zip
CL.zip
CM.zip
CN.zip
CO.zip
CR.zip
CS.zip
CU.zip
CV.zip
CW.zip
CX.zip
CY.zip
CZ.zip
DE.zip
DJ.zip
DK.zip
DM.zip
DO.zip
DZ.zip
EC.zip
EE.zip
EG.zip
EH.zip
ER.zip
ES.zip
ET.zip
FI.zip
FJ.zip
FK.zip
FM.zip
FO.zip
FR.zip
GA.zip
GB.zip
GD.zip
GE.zip
GF.zip
GG.zip
GH.zip
GI.zip
GL.zip
GM.zip
GN.zip
GP.zip
GQ.zip
GR.zip
GS.zip
GT.zip
GU.zip
GW.zip
GY.zip
HK.zip
HM.zip
HN.zip
HR.zip
HT.zip
HU.zip
ID.zip
IE.zip
IL.zip
IM.zip
IN.zip
IO.zip
IQ.zip
IR.zip
IS.zip
IT.zip
JE.zip
JM.zip
JO.zip
JP.zip
KE.zip
KG.zip
KH.zip
KI.zip
KM.zip
KN.zip
KP.zip
KR.zip
KW.zip
KY.zip
KZ.zip
LA.zip
LB.zip
LC.zip
LI.zip
LK.zip
LR.zip
LS.zip
LT.zip
LU.zip
LV.zip
LY.zip
MA.zip
MC.zip
MD.zip
ME.zip
MF.zip
MG.zip

In [3]:
data=pd.read_csv('files/AD.txt',sep='\t',header=None)

In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2986043,Pic de Font Blanca,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.64991,1.53335,T,PK,AD,NaN,00,NaN,NaN,NaN,0,NaN,2860,Europe/Andorra,2014-11-05
1,2993838,Pic de Mil-Menut,Pic de Mil-Menut,Pic de Mil-Menut,42.63333,1.65000,T,PK,AD,"AD,FR",B3,09,91,9024,0,NaN,2138,Europe/Andorra,2014-11-05
2,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,MT,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2803,Europe/Andorra,2014-11-05
3,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2685,Europe/Andorra,2014-11-05
4,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663,66146,0,NaN,2411,Europe/Andorra,2014-11-05


In [5]:
names=[]
names.append('id')
names.append('name')
names.append('ascii_name')
names.append('alternate_names')
names.append('lat')
names.append('long')
names.append('feature') # feature class http://www.geonames.org/export/codes.html
names.append('feature code')
names.append('country')
names.append('country_alt') # weird other country something
names.append('admin1')
names.append('admin2')
names.append('admin3')
names.append('admin4')
names.append('population')
names.append('elevation')
names.append('elevation_dem')
names.append('timezone')
names.append('mod') # Date modified

In [6]:
data.columns=names

In [7]:
data.head()

,id,name,ascii_name,alternate_names,lat,long,feature,feature code,country,country_alt,admin1,admin2,admin3,admin4,population,elevation,elevation_dem,timezone,mod
0,2986043,Pic de Font Blanca,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.64991,1.53335,T,PK,AD,NaN,00,NaN,NaN,NaN,0,NaN,2860,Europe/Andorra,2014-11-05
1,2993838,Pic de Mil-Menut,Pic de Mil-Menut,Pic de Mil-Menut,42.63333,1.65000,T,PK,AD,"AD,FR",B3,09,91,9024,0,NaN,2138,Europe/Andorra,2014-11-05
2,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,MT,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2803,Europe/Andorra,2014-11-05
3,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2685,Europe/Andorra,2014-11-05
4,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663,66146,0,NaN,2411,Europe/Andorra,2014-11-05


### Grab admin 1 names

In [115]:
!wget http://download.geonames.org/export/dump/admin1CodesASCII.txt

--2015-10-15 15:05:14--  http://download.geonames.org/export/dump/admin1CodesASCII.txt
Resolving download.geonames.org (download.geonames.org)... 188.40.33.19
Connecting to download.geonames.org (download.geonames.org)|188.40.33.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137197 (134K) [text/plain]
Saving to: ‘admin1CodesASCII.txt’

100%[======================================>] 137,197      142KB/s   in 0.9s   

2015-10-15 15:05:16 (142 KB/s) - ‘admin1CodesASCII.txt’ saved [137197/137197]



In [8]:
admin1=pd.read_csv('admin1CodesASCII.txt',sep='\t',header=None,names=['code','name','ascii_name','pop'])

In [9]:
admin1.head()

,code,name,ascii_name,pop
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131
3,AD.03,Encamp,Encamp,3040684
4,AD.02,Canillo,Canillo,3041203


In [10]:
admin1['country']=admin1.code.apply(lambda x:x.partition('.')[0])
admin1['admin_code']=admin1.code.apply(lambda x:x.partition('.')[2])

In [11]:
admin1.head()

,code,name,ascii_name,pop,country,admin_code
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162,AD,06
1,AD.05,Ordino,Ordino,3039676,AD,05
2,AD.04,La Massana,La Massana,3040131,AD,04
3,AD.03,Encamp,Encamp,3040684,AD,03
4,AD.02,Canillo,Canillo,3041203,AD,02


### Create Tables

In [12]:
conn = pymysql.connect(host='localhost',user='root',passwd=PW, db='geo',charset='utf8')
cur = conn.cursor()

In [44]:
res=cur.execute('create table admin1 (name varchar(100) character set utf8, ascii_name varchar(100) character set utf8, country varchar(2),admin varchar(10))',)

In [48]:
res=cur.execute('create table places (name varchar(100) character set utf8,clean_name varchar(100) character set utf8, lat float,lon float,country varchar(2),pop int, elevation mediumint)',)

### Add rows for admin 1

In [30]:
cmd="INSERT INTO admin1 (ascii_name,country,admin) VALUES (%s,%s,%s)"

In [19]:
admin1['name'].fillna("",inplace=True)

In [20]:
admin1['admin_code']=admin1['admin_code'].apply(str)

In [28]:
admin1.head()

,code,name,ascii_name,pop,country,admin_code
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162,AD,06
1,AD.05,Ordino,Ordino,3039676,AD,05
2,AD.04,La Massana,La Massana,3040131,AD,04
3,AD.03,Encamp,Encamp,3040684,AD,03
4,AD.02,Canillo,Canillo,3041203,AD,02


In [46]:
for row in admin1.iterrows():
    temp=(row[1]['ascii_name'],row[1]['country'],row[1]['admin_code'])
    res=cur.execute(cmd,temp)
    conn.commit()

### Add rows for places

In [24]:
cmd="INSERT INTO places (name,lat,lon,country,pop,elevation) VALUES (%s,%s,%s,%s,%s,%s)"

In [25]:
for row in data.iterrows():
#    print row[0],row[1]
    temp=(row[1]['name'],row[1].lat,row[1]['long'],row[1].country,row[1].population,row[1].elevation_dem)
    print temp
    
    res=cur.execute(cmd,temp)
    conn.commit()
    
    assert cur.rowcount==1

('Pic de Font Blanca', 42.64991, 1.53335, 'AD', 0, 2860)
('Pic de Mil-Menut', 42.63333, 1.65, 'AD', 0, 2138)
('Roc M\xc3\xa9l\xc3\xa9', 42.58765, 1.7402799999999998, 'AD', 0, 2803)
('Pic des Langounelles', 42.61203, 1.47364, 'AD', 0, 2685)
('Pic de les Abelletes', 42.52535, 1.73343, 'AD', 0, 2411)
('Estany de les Abelletes', 42.52915, 1.7336200000000002, 'AD', 0, 2260)
('Port Vieux de la Coume d\xe2\x80\x99Ose', 42.625679999999996, 1.6182299999999998, 'AD', 0, 2687)
('Port de la Cabanette', 42.6, 1.73333, 'AD', 0, 2379)
('Port Dret', 42.60172, 1.4556200000000001, 'AD', 0, 2660)
('Costa de Xurius', 42.50692, 1.47569, 'AD', 0, 1839)
('Font de la Xona', 42.55003, 1.44986, 'AD', 0, 1976)
('Xixerella', 42.55327, 1.48736, 'AD', 0, 1417)
('Xixerella', 42.55294, 1.48764, 'AD', 0, 1417)
('Riu Xic', 42.57165, 1.6755400000000003, 'AD', 0, 1851)
('Pas del Xic', 42.497659999999996, 1.57597, 'AD', 0, 1669)
('Roc del Xeig', 42.56068, 1.4898, 'AD', 0, 1643)
('Collada del Xeig', 42.57512, 1.49009, 'AD'

KeyboardInterrupt: 

In [93]:
cmd="INSERT INTO places (name,lat,lon,country,pop,elevation) VALUES (%s,%s,%s,%s,%s,%s)"

In [94]:
res=cur.execute(cmd,("test",99.9,99.9,"TE",999,999))

In [ ]:
res=cur.execute()

In [48]:
cmd="INSERT INTO `places` (`name`) VALUES (%s)"

In [95]:
conn.commit()

In [96]:
cur.rowcount

1